# Milestone 3 - Report
------
In this notebook we are going to summarize what was done in Milestone 2 and explain all that was done for the final Milestone. The focus at this point of the project was placed on constructing the awareness model using our different metrics and the final datastory. 

In [ ]:
import os
import json
import time
import folium
import branca
import jenkspy
import pickle
import unicodedata
import collections
import numpy as np
import pandas as pd
from scipy import spatial, stats
import matplotlib.pyplot as plt
import seaborn as sns
from pygsp import graphs, filters, plotting
from IPython.display import Image

%matplotlib inline

## 0. Abstract

*What's the motivation behind your project? A 150 word description of the project idea, goals, dataset used. What story you would like to tell and why?*

Major events happen on a regular basis all around the world, some involving high number of casualties but the resulting reaction on the international scale is often far from proportional. Most of the time the largest reaction comes from the place where the incident occurred or places which are closeby. The objective would be to create an awareness map, and determine why people react to an event. From that we would attempt to define an awareness metric. We want to see how factors other than physical proximity come into play such as country, culture, language, religion. With this we could determine which country has the highest level of international awareness. The project would require the Twitter API to acquire hashtag specific tweets with geolocation and therefore measure the awareness and reactions of different communities to a given event. GDELT would be used to recover standardised information regarding different events.
____
____
____


# 1. Recapitulation of Milestone 2

To recap what was done in the previous milestone, here we present a brief summary of the previous notebook which can be found here **(TODO : mettre le lien)**. 

____

## 1.1 Tweet Acquisition
A lot of time was spent acquiring the Tweets for two main reasons : 
- The Twitter dump did not contain the geolocations of the tweets or the users and only containecd 10% of all tweets for a given period. Given that users to not necessarily give their location we wanted to avoid cases where we would not have enough overall located tweets. 
- The Twitter API does not permit the recovery of tweets which are over 1 week old when searching for tweets with a given hashtag around a specific date. 

That is why a significant amount of time was put into recovering the tweets through webscraping using phantom js with random scrolling time and random breaks to avoid being blocked by Twitter.

The tweets were acquired based on their hashtags, all the while making sure to have a wide range of hashtags in different languages. 

**TODO : put the link towards the file containing all the hashtags**

____

## 1.2. Tweet Geolocalisation

Once the tweets were recovered we only kept those for which we had either the geolocalisation of the tweets itself or of the user. As the locations are manually input, they are not standardized and the corresponding location needed to be determined. 

The first step was therefore to format the strings by removing all special characters as setting everything to lower case for example.  Afterwards, we created multiple mappings which were used in the order presented as they represent the confidence in the given method : 
- A country mapper which uses multiple variations of the country name (different spellings and languages) to determine the country it corresponds to with the given ISO2 code
- A capital mapper which maps the capital of a country to the ISO2 code
- A first city mapper where duplicates were removed as there are multiple cities in the world with the same name (example : Torronto in Canada, the US and Australia). 
- A final city mapper with all the cities of all the countries in the world using the database provided here **(TODO : mettre le lien)**

** TODO : put the image of the bird with the number of tweets** 


## 1.3. Visualization of the Reaction For An Event

We then observed for one of the selected events the number of related tweets on a Chloropleth map. This helped us note certain issues in the mappings (high number of tweets in Antarctica for example). 

____


## 1.4. Enriching the Data 

To create our awareness model we wanted to recover information which could be used to determine a sort of distance between the different countries, that is why we recovered for all countries: 
- area
- ISO2
- ISO3
- languages
- border countries
- latitude and longitude
- language_codes
- Internet users
- population
- GDP
- GDP per capita
- religions
- government types
- population in poverty
- unemployment rate

As you can see, we have more features than we had originally presented in Milestone 2. That's because we realized that one of the datasets we used had a lot more (and more complete) information that was of our interest. After all, the 4 datasets that we were using originally ended up being just two, which are these:

- Dataset 1: https://github.com/mledoze/countries
- Dataset 2: https://github.com/opendatajson/factbook.json

To see all the detailed process of the extraction of the data see "Data enriching.ipynb".
____
____
____


____
____
____

# 2. Improving the Mappings

One main issue with the mapping up until now is that there were redundancies which were not correctly handled. At the time of Milestone 2 we would just go through the complete mapping sequentially and stop at the first match. As there were duplicates there was no guarantee that the city found was the most coherent. That is why we corrected the mapping to take into account the population of the different cities which was also provided in the GeoNames database http://download.geonames.org/export/dump/. Therefore when there are multiple cities with the same name in different countries, we only keep the city with the largest population as it would be the most probable. 

We also replaced our intermediate city mapping with a mapping containing the top 20'000 cities in the world (and therefore the most probable). With appoximately 200 countries this corresponds to around 100 cities per country. 

# 3. Metrics For the Awareness Model

To create our model we originally planned to construct a graph linking all countries where the weights of the segments would represent the proximity between countries in terms of awareness to major events. 

For that we used the dataset containing all the information regarding the countries and looked to give numeric values to all the different categories mentioned previously and a few new ones: 

1. Languages
2. Governments 
3. Distances between countries :
    - Birds eye view distances 
    - Hop matrix : the number of countries which separate two given countries (**TODO : noter quil y a le souci des distances a traverser au sein des pays qui n'est pas pris en compte, exemple de la Finlande a la Chine qui passe uniquement par la russie**). 
    - Normalized Adj matrix : a matrix which gives the neighboring countries with a normalizing factor to account for multiple countries on the border
    - Flight routes between countries
4. Religion
5. Population, Area, GDP, Internet Users etc...


What needed to be taken into account here is that we have many variables which are categoric. Therefore we needed to find a way to attribute numeric values in order to construct the final graph.

Remark : most of the information was extracted from the CIA factbook as it is a complete and realiable source.  **TODO : put the link**

**TODO / REMARQUE A GARDER EN TETE POUR LA CONCLUSION AU PIRE : voir si le fait que le pays soit très grand avec une grande population qui peut diminuer l'intérêt par rapport à ce qui se passe ailleurs dans le monde**

____

## 3.1. Languages

Languages are representative of culture and most of the time, countries which speak the same language have a given proximity. That is why this metric was used to create the graph. 

Two main possibilities presented themselves :
- binary : if two countries have the same official language then they are linked
- more informed : some languages are similar in the sense of understanding the other language, learning it, etc. That is why we wanted to take into account Linguistic distances. Unfortunately this is a current research topic and there is no database which gives the Linguistic distances between all the official languages of all countries. Certain databases exist linking English to other languages but this is not sufficient for our application. That is why we decided to use another metric, albeit more abstact, which is to say the phylogenetic trees linking languages together. 


Using the phylogenetic trees taken here (**TODO insert nice link**) http://glottolog.org/glottolog/family, we were able to construct a distance metric and compute the distances between all languages as can be seen below. The notebook handling this can be found here (**TODO : mettre le lien du notebook**)

In [ ]:
# TODO METTRE LA MATRICE & HEATMAP DE DISTANCE ENTRE LES LANGUES

From this language distance matrix we were able to compute a standardized distance between all countries. 

The idea is the following :
- if countries have an official language in common, set the distance to 0
- else get the distances between the official languages and compute the average of the non infinite distances. 
- else set the distance to inf


In [ ]:
# TODO METTRE LA MATRICE DISTANCE / HEATMAP ENTRE LES PAYS SELON LA LANGUE

The issue with this type of distance is that it takes discrete values which don't account for the actual distance between the different languages in terms of comprehension, facility to learn the other language etc.. For example the distance between French and a dialect ....
**TODO : afficher deux valeurs de distance similaires mais qui ne sont pas représentatifs**

Results more representative of the reality of the situation could have been obtained had we had not only the phylogenetic links between the languages but the moment that the a new language appeared from a mother language. Incoroporating this time component would have given a more precise approximation, assuming that languages evolve at similar rates and that once the new language has emerged that there is minimal contact / influence from the mother or any sister languages. 

____

## 3.2. Governments

The type of government is also a good representative of the people's perception to worldwide events. People of different countries with the same government type will be influenced in a similar way. 

After getting the list of all the government types (see "Data enriching.ipynb" for more details on how we got them) we have to convert them to a numeric scale with a meaning.

First, we start by dividing all the government types in three groups and assigning them numerical values -1, 0 and 1. These are the resulting groups:
- Group 1:
    - 'parliamentary democracy'
    - 'parliamentary republic'
    - 'federal republic'
    - 'federation of monarchies'
    - 'semi-presidential republic'
    - 'semi-presidential federation'
- Group 2: 
    - 'non-self-governing overseas territory'
    - 'in transition'
    - 'unknown'
- Group 3: 
    - 'presidential republic'
    - 'presidential democracy'
    - 'monarchy'
    - 'theocratic republic'
    - 'communist state'
    - 'absolute monarchy'

As our criteria to group the government types, we used the variable of the power that the leaders of the government have. Coutries where the leader has a lot of power go in group 3 and will be assigned to the value of 1, the others in group 1 and they will be assigned the value -1. For the non well defined government types, we placed them at the center of the scale with the value of 0.

Here is the actual implementation (full code on "Data enriching.ipynb"):

In [ ]:
# Reading the dataframe
data = pd.read_pickle('./DataEnriching/data.pickle')

# Defining the three groups
gov_type_array = [['parliamentary democracy', 'parliamentary republic', 'federal republic', 'federation of monarchies',
                   'semi-presidential republic', 'semi-presidential federation'],
                  ['non-self-governing overseas territory', 'in transition', 'unknown'],
                  ['presidential republic', 'presidential democracy', 'monarchy', 'theocratic republic', 'communist state',
                   'absolute monarchy']]

# Creating the mapping dictionary
mapping_gov_type=dict()
for i, gov_group in enumerate(gov_type_array):
    for gov in gov_group:
        if i==0:
            mapping_gov_type.update({gov: -1})
        elif i==1:
            mapping_gov_type.update({gov: 0})
        else:
            mapping_gov_type.update({gov: 1})

# Mapping gov_type values to their numerical value (1, 0, -1)
data['gov_type_num'] = data.gov_type.map(mapping_gov_type)
data[['gov_type', 'gov_type_num']].head()

In order to compute the distance metric between government types, the numeric government type will be used. The distance metric will be defined by the absolute value of the subtraction between the values of each pair of countries. That way, we end up with a symetric metric with possible values of 0, 1 and 2.

In [ ]:
# Reading the dataframe and setting ISO2 as index (as it will be the useful country code)
data = pd.read_pickle(os.path.abspath(os.path.join(os.getcwd() + '\DataEnriching\data.pickle')))
data.reset_index(inplace=True)
data.set_index('ISO2', inplace=True)

In [ ]:
# Selecting numeric government type column
gov_type_df = data[['gov_type_num']]

# Creating the matrix with the distances
gov_distance_df = pd.DataFrame(columns=gov_type_df.index.tolist())

for country1, value1 in zip(gov_type_df.index.tolist(), gov_type_df['gov_type_num']):
    row = []
    for country1, value2 in zip(gov_type_df.index.tolist(), gov_type_df['gov_type_num']):
        row.append(abs(value1-value2))
    
    
    dictionary = dict(zip(data.index.tolist(), row))
    gov_distance_df = gov_distance_df.append(dictionary, ignore_index=True)
    
gov_distance_df.index = gov_type_df.index
gov_distance_df.head(10)

____

## 3.3. Distances 

Distance tends to play an important role in determining people's interest regarding a specific event. But there are several types of distances which can be considered when speaking of countries. 

The first and most evident is the **birds-eye-view distance** between the countries. This is computed based on the positions of the center of each country. But this is not enough. Take for example the US and Canada, the distance between the center of thees two countries is much larger than the distance between France and any of its neighboring countries. 

So what else could we consider to have a more representative representation of distance between the different countries? We came up with three other distance metrics which combined would be more complete.


1. **Relative Importance of Neighbors** :  To take into account the fact that two countries can be neighbors and still have a big distance between them we decided to create a metric which would give importance to countries which are direct neighbors. We also wanted to make sure to give each neighbor the importance it is due. For example France has  multiple countries at its border. But that does not mean that each of these countries are of similar importance. That is why we weighted this metric by the size of the countries. Therefore a small neighboring country such as Luxemburg would have a smaller weight than Germany for example.  

2. **Hop Distance** : This second metric accounts not only for direct neighbors but also for the smallest number of countries which would need to be traversed to connect any two countries in the world. 

3. **Flight Routes** : This last metric accounts for movement of populations between the different countries. The assumption is that the existence of flight routes is due to the fact that people exhibit a certain interest for the other country. The more often you visit a place, the more likely you are to be interested in what is going on there. 


These metrics are all constructed in the country distances noteboook in the GeoMetrics folder (**TODO : put link **). Refer to this noteboook for more detail on how the metrics were constructed. 

** Bird's Eye View Distance / Real Distance **

Constructing this mapping was relatively straightforward. Using the position in latitude and longitude of the countries the distance between all countries in computed all the while being carefull with the extreme values of longitude. It is important to consider the shortest path as the earth is round. 

** TODO : load the pickle file and plot the heatmap as well as the distribution of values in KILOMETERS! (factor 105)**


** Hop Distance **


For this metric as well as for the following we established an adjacency matrix of the connected countries in term of borders. It is important to note that Islands and certain continents are isolated which means that the resulting graph will not be fully connected. 

This will also be a usefull tool to create a neigborhood influence matrix.
We can visualize the graph using networkx. 

**TODO : put the graph from Adrian's notebook with the nodes placed in the position of the countries**

With the obtained graph in networkx we compute the shortest path between all pairs of nodes to obtain what we call the hop matrix between all countries. Countries which are not connected have a hop distance of infinity between them in the final dataframe. 

** TODO : insert the heatmap of the final hop matrix**

**TODO : put a screenshor of the map for a given country + LINK to the html with the map**



** Relative Importance of Neighbors **

The adjacency matrix established previously is then used to create a weighted adjacency matrix where the edges weighted by the size of the neighboring country over the sum of the sizes of all the neighbors. The graph is no longer an undirected graph. 

**TODO : put a screenshot of the map for a given country and link **


** Flight Routes **

Given a dataset of 59036 routes between 3209 airports on 531 airlines in the world as of January 2012. In addition the that we are able so associate all the Airports to their country thanks to a second dataset. **TODO : put the links towards the datasets**

Once that was done, we associated all the routes with the departure and arrival countries. That way were able to determine for each country the proportion of flights to all other countries. 

In the same way as before the graph is a directed graph.

**TODO : put the heatmap**

**TODO : put the graph with the nodes at the positions of the countries**

**TODO : put a screenshor of the map for a given country**

**TODO : put the link towards the interactive map for all the distance metrics**

____

## 3.4. Religions 

Religion, like government, affects the way of thinking of the people. Religion bonds between countries will definitely have an impact on the awareness of events that happen in countries with the same religions.

Here we can see the religion data that we were originally using in Milestone 2:

In [ ]:
rel_df_milestone2 = pd.read_pickle("./DataEnriching/Pickles for Milestone 3/final_rel_df.pickle")
rel_df_milestone2.head()

Unfortunately, there were too many countries for which the religion information was not complete. That is why we went back to the factbook which was more complete. 

### Methodology
From now on, we will explain how we took the data from the factbook dataset.

We could extract the religions from Dataset 2, but we had to do some cleaning to have the desired format of the data. First we needed to know which where the religions that were in the dataset, so we created an array containing the unique names of the religions that appeared througouth each country. Once we know all the religions, we created the religions dataframe, containing the percentage of every religion in every country. Finally, we grouped the small religions in 10 broader categories of religions.

As said, we start by creating the unique religions array. Here is a sample of part of the array:

In [ ]:
#unique_religions[:20]
['Adventist', 'Animist', 'Armenian Apostolic', 'Assembly of God', 'Awakening Churches/Christian Revival', 'Badimo',
 "Baha'i", 'Baptist', 'Bektashi', 'Buddhism', 'Buddhist', 'Bukot nan Jesus', 'Calvinist', 'Cao Dai', 'Catholic', 'Christian',
 'Christianity','Church of England', 'Church of Ireland', 'Church of Norway']

Now that we have all possible religions in one array, we will create the dataframe (rel_df), with all religions as columns, and the country as the index. The cell values will represent the percentage of the religion in the country.

Some of the countries had bad formated data to be extracted (i.e. percentages given by ranges like 10-30%, some countries didn't have the percentages, etc). We manually added the rows of the countries that had issues. For the countries that didn't have percentages, we searched on Wikipedia for the percentages and we also added them manually.

To create the rel_df dataframe, the rows were appended into an initial empty dataframe. The rows were created by looping through all unique_religions and checking if the country in particular had that religion. If it had it, we inserted the percentage. If it didn't have it, we inserted the value zero in that religion column.

Here is what the rel_df looks like, after dropping the smallest religions (columns) that didn't have more than 10% in any country:

In [ ]:
rel_df = pd.read_pickle('./DataEnriching/rel_df.pickle')
rel_df.set_index('ISO2', inplace=True)
rel_df.head()

### Grouping religions under broader categories
We grouped the religions in broader categories by adding the percentages. We had to dropp some columns that didn't provide any information about the religious similarity between two countries (i.e unspecified, none or other, other, etc).

In [ ]:
# Here are the broader categories that we came up with
protestants = ['Calvinist', 'Church of Norway', 'Congregational Christian Church', 'Ekalesia Niue', 'Evangelical', 
               'Evangelical Lutheran', 'Evangelical Lutheran Church of Iceland', 'Evangelical or Protestant', 'Lutheran',
               'Protestant', 'Protestant and other', 'Seventh-Day Adventist', 'non-Catholic Christians', 'Armenian Apostolic', 
               'Assembly of God', 'Christian', 'Kimbanguist', 'Mormon', 'Zionist Christian', 'nondenominational']
               
catholic = ['Awakening Churches/Christian Revival', 'Catholic', 'Roman Catholic', 'nominally Roman Catholic']
orthodox = ['Eastern Orthodox', 'Ethiopian Orthodox', 'Greek Orthodox', 'Macedonian Orthodox', 'Orthodox', 'Orthodox Christian',
                'Russian Orthodox', 'Serbian Orthodox']
buddhism = ['Buddhism', 'Buddhist', 'Lamaistic Buddhist']
hindu = ['Hindu', 'Indian- and Nepalese-influenced Hinduism']
jewish = ['Jewish', 'Zionist', ]
muslim = ['Muslim', 'Sunni Muslim']
oriental = ['Shintoism', 'Taoist', 'mixture of Buddhist and Taoist']
other = ['Vodoun', 'eclectic mixture of local religions', 'folk religion', 'indigenous beliefs']
animist = ['animist', 'animist or no religion']
atheist = ['atheist or agnostic', 'no religion', 'non-believer/agnostic', 'non-believers']
unaffiliated = ['unaffiliated', 'unaffiliated or other']

# Those religions will be dropped
dropped_cols = ['Kempsville Presbyterian Church', 'none', 'none or other', 'other',
                'other Christian', 'other and unspecified', 'other or none', 'other or unspecified', 'unspecified',
                'atheist and agnostic']

# Arrays needed to run next cell
final_categories = [protestants, catholic, orthodox, buddhism, hindu, jewish, muslim, oriental, other, animist,
                   atheist, unaffiliated]
final_categories_names = ['protestants', 'catholic', 'orthodox', 'buddhism', 'hindu', 'jewish', 'muslim', 
                           'oriental', 'other', 'animist', 'atheist', 'unaffiliated']

The function that unifies the small religions to broader religion groups, and the resulting dataframe are the following:

In [ ]:
def generate_categories_df(dataframe):
    df = pd.DataFrame()
    for category, category_name in zip(final_categories, final_categories_names):
        df[category_name] = dataframe[category].sum(axis=1)
    return df

categorized_df = generate_categories_df(rel_df)
categorized_df.head()

To compute the distances, the euclidean dot product was applied using the following function, that returns the matrix with the distances between each pair of countries.

In [ ]:
rel_distances = spatial.distance.squareform(spatial.distance.pdist(categorized_df,'euclidean'))

In [ ]:
# Converting it to a dataframe
rel_distance_df = pd.DataFrame(rel_distances, index=rel_df.index.tolist(), columns=rel_df.index.tolist())
rel_distance_df.head(10)

**TODO : insert heatmap of the distances**

**TODO INSERT MAP WITH ALL RELIGIONS**

In the previous cell we can see the visualization of the religion metric on a map. When clicking on countries which are majoritarely christian the distance metric does not account for the proximity between protestant, catholic and orthodox. That is why we decided to merge the three religions (protestants, catholics, orthodox) into one group (christianity). The results can be seen in the following map. 

**TODO INSERT MAP WITH CHRISTIANS TOGETHER**

Here we can see that now two countries majoritarely christians are closer in the religion distance metric, meaning that we solved the problem.


To compute the last map, we had to solve also the issues with the lack of data of some countries. In the categorized_df dataframe there are countries that have null values in every religion. This represents a big issue because when computing the distance between countries, these countries will have distance zero not for their religious proximity, but because we lacked the data. We managed to find the percentages of the main religions in most of the important countries by searching in Wikipedia. Here is an example of some manually inserted values:

In [ ]:
# We found the percentages on wikipedia by doing the search on google: percentages of main religions [country]
data.set_value('Greenland', ('religion', 'christianity'), 0.96);
data.set_value('Madagascar', ('religion', 'atheist'), 0.84);
data.set_value('North Korea', ('religion', 'atheist'), 0.643);
data.set_value('Palestine', ('religion', 'muslim'), 0.85);
data.set_value('Palestine', ('religion', 'jewish'), 0.12);
data.set_value('Sudan', ('religion', 'muslim'), 0.97);

____

## 3.5. Population, Area, GDP, Internet users, population in poverty, unemployment rate

Countries can be characterized by general attributes such as population, size, gross domestic product (GDP), poverty line, internet users and so forth. These metrics are interesting because they are in direct link to the number of tweets in a conutry and can be used as normalizing factors. 
         
As we announced previously, by using the factbook dataset for Milestone 3 we were able to extract more features than in Milestone 2. These features are straight forward to extract from Dataset 2 (see "Data enriching.ipynb"). From the data.pickle file we can extract them directly. The result is the following:

In [ ]:
other_features = pd.read_pickle('.\DataEnriching\data.pickle')
cols = [('POP',''),('area',''),('gdp',''),('gdp_capita',''),('Internet users',''),('pop_pov',''),('unemployment','')]
other_features = other_features[cols]
other_features.head()

____

##  3.6. Estimating Number of Active Tweeters 

As will be explained in the next portion of the notebook it is important to have a normalizing factor to account for differences in twitter activity between the different country. The ideal thing would have been to have the number of active tweeters per country. A second best would have been to simply have the number of tweeters per country. Unfortunately none of this is available to the public. That is why it was important to create our own estimation of these values to generate the model. 

We had at our disposal the number of tweets worldwide for 8 different events. We used this to compute the average number of tweets per country and used it as a baseline. We were careful to compute the average of tweets for a given country only using the data when there was no event in the given country. In clearer terms we did not take the number of tweets in France during Charlie Hebdo to compute the average number of tweets. We relied on the 7 other events. In the same fashion we did not consider the tweets for france during the event in Belgium as the countries are sufficiently close to generate larger than normal reactions. The inverse was also done for Belgium.  This implies that we compute the average number of tweets (baseline) using only 6 events for France and Belgium versus 7 for the others. 


The code used to compute the baseline number of tweets over a week called all_tweets can be found at the start of the Correlations and Linear Regression Model notebook. Here we show a modified excerpt as well as the difference when computing the mean without the values in the countries where the event happened. 


In [ ]:
import os
events = [x for x in os.listdir('../../Project Data/Tweets/') if "DS_Store" not in x]

for ignore in [False, True]:
    
    all_tweets = list()
    for event in events:
        all_tweets.append(pickle.load(open('../../Project Data/Tweets/{}/Geocoded/summary.pickle'.format(event), 'rb')))

    if ignore:
        print()
        print("Ignoring number of tweets for countries in which the event occurred")
        # Setting values to nan if the event occured in the country
        for idx, country in enumerate(["BE", "FR", "TR", "LB", "NG", "NG", "US", "PK"]):
            if country== "BE":
                all_tweets[idx].loc["FR",:] = np.nan
            if country== "FR":
                all_tweets[idx].loc["BE",:] = np.nan
            if country == "US":
                all_tweets[idx].loc["BE",:] = np.nan
                all_tweets[idx].loc["FR",:] = np.nan
            all_tweets[idx].loc[country,:] = np.nan
    else :
        print("Computing baseline using all data")
    all_tweets = pd.concat(all_tweets, axis=1).mean(axis=1, skipna = True)
    print("United States : ", int(all_tweets["US"]))
    print("France : ", int(all_tweets["FR"]))
    print("Belgium : ", int(all_tweets["BE"]))
    print("Pakistan : ", int(all_tweets["PK"]))
    print("Turkey : ", int(all_tweets["TR"]))
    print("Nigeria : ", int(all_tweets["NG"]))
    print("Lebanon : ", int(all_tweets["BE"]))
        

We can clearly see that there is a difference in terms of baseline tweets for all these countries, the most flagrant being the factor 35 and 28 of difference for France and Belgium respectively. This also shows the level of reaction which occurred in both countries. 

____

# 4. Constructing the Awareness Model 

Given all of these metrics we wanted to creat a graph linking all countries. The more countries have attributes in common, the more likely they are to react to what is going on. But how do we quantify that two countries share things in common? 

We first observed whether any variables correlated highly with the measured reactions. Unfortunately as we will show, it was not the case for any of the events. This can be seen in the [Correlations and Linear Regression Model notebook](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/Correlations%20and%20Linear%20Regression%20Model.ipynb)

Our original idea was to construct a graph linking the different countries. We wanted to use what we saw in recommender systems and topic detection would better be able to select features which best link the countries together. The issue with this method is that it is difficult to determine why the outputs were not optimal. This can be seen in the [Matrix Factorisation - LDA Model](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/Correlations%20and%20Linear%20Regression%20Model.ipynb) notebook  


That is why we attempted two alternative methods :
- Using random search to find the optimal weights combining our features to construct a final graph upon which we could do diffusion and compare the output to the reactions for the different events. See the [Weighting Metrics and Graph Diffusion](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/Weighting%20Metrics%20and%20Graph%20Diffusion.ipynb) notebook. 

- Using support vector regression which is a non linear regression method to predict the reaction directly [SVR Model](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/SVR%20Model.ipynb) 
       
The issue with both of these methods is that we have to predict the outputfor 250 countries with a very limited dataset (8 events). We would have required a much larger dataset consisting of at least 1000 events in order to properly train and test the model.  


## 4.1. Correlations

The first thing we wanted to do was to verify whether any of the our metrics highly correlated with the observed reactions. That is what we did in the Correlations and Linear Regression Model notebook [Correlations and Linear Regression Model notebook](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/Correlations%20and%20Linear%20Regression%20Model.ipynb)

Originally we had hoped that a subset of features would correlate with the reaction for a given number of events. Unfortunately this was not the case for any of the observed events. 

Loading the important dataframes

In [ ]:
features_df = pd.read_pickle(os.path.join("AwarenessModel","correlation_features.pkl"))
all_tweets = pd.read_pickle(os.path.join("AwarenessModel","all_tweets.pickle"))


data = pickle.load(open('DataEnriching/data.pickle','rb'))
data = data.reset_index()
data = data.set_index("ISO2")

# Keep only relevant fields 
data = data[['ISO2','POP', 'area', 'gdp', 'gdp_capita', 'pop_pov', 'region']]

lang_feature_df = pd.read_pickle('LinkingLanguages/stand_country_dist_languages.pkl')
fligh_routes_df = pd.read_pickle('GeoMetrics/flight_routes.pickle')
hop_distance_df = pd.read_pickle('GeoMetrics/hop_distance.pickle')
neighbor_influence_df = pd.read_pickle('GeoMetrics/neighbor_influence_v2.pickle')
dist_feature_df = pd.read_pickle('GeoMetrics/real_distance.pickle')
religion_and_more_df = pd.read_pickle('DataEnriching/Pickles for Milestone 3/6_feature_df.pickle')

# Load the number of internet users
internet_users = pickle.load(open('AwarenessModel/internet_users.pickle','rb'))
internet_users.set_index("ISO2", inplace = True)
internet_users.replace("unknown", np.nan, inplace = True)
internet_users["Internet users"] = internet_users["Internet users"].astype(np.float)
med = np.median([x for x in internet_users["Internet users"].tolist() if not np.isnan(x)])
internet_users = internet_users["Internet users"].apply(lambda x: x if not(np.isnan(x)) else med)


Here are the set of functions used to process a given event and plot the correlations as well as a linear regression model.

In [ ]:
def load_tweets(event, country_code):
    plt.figure(figsize = (15,10))
    # Loading the Number of tweets of the event in turkey per country
    summary_tweets = pickle.load(open('../../Project Data/Tweets/{}/Geocoded/summary.pickle'.format(event), 'rb'))
    #plot = sns.barplot(summary_tweets.index,summary_tweets.text)
    return summary_tweets


def event_specific_features(country_code, summary_tweets):
    # Create the dataframe for the given country
    country_features_df = pd.concat([lang_feature_df.loc[:,("language_dist",country_code)],
                           fligh_routes_df.loc[:,("flight_routes",country_code)],
                           hop_distance_df.loc[:,("hop_distance",country_code)],
                           neighbor_influence_df.loc[:,("neighbor_influence",country_code)],
                           dist_feature_df.loc[:,("real_distance",country_code)], 
                           summary_tweets.loc[:,"text"],
                           data.loc[:,["area", 'gdp', 'gdp_capita', "POP", "2016_gdp_total", "region"]],
                           features_df.loc[:,("Normalized internet users")],
                           features_df.loc[:,("Internet users")]],axis=1)
    #features_df.sort_index(axis=1,inplace=True)

    # Rename the columns
    country_features_df.columns = ['language_dist', 'flight_routes', 'hop_distance', 'neighbor_influence',\
                                   'real_distance', 'text', 'POP', 'area', 'gdp', 'gdp_capita',\
                                    'region', 'Normalized internet users', 'Internet users']
    
    # Change the type of the given columns from object to float
    country_features_df.flight_routes = country_features_df.flight_routes.astype(np.float)
    country_features_df.neighbor_influence = country_features_df.neighbor_influence.astype(np.float)
    
    #country_features_df["text"] = np.log(country_features_df["text"]+1)
    
    country_features_df["Normalized Tweets"] = (country_features_df["text"]-all_tweets) / (all_tweets+1)


    # Remove the information pretaining to the concerned country
    country_features_df.drop([country_code], inplace = True)
    #country_features_df.drop(["BE", "FR"], inplace = True)
    return country_features_df

# Plot correlations between the different metrics and the 
# normalized number of tweets. Compute the regression + R squared metric

def plot_correlations_with_regression(country_features_df):
    fig = plt.figure(figsize = (15,20))

    y_all = country_features_df.loc[:,'Normalized Tweets'].tolist()

    # Compute and plot the correlations for the first columns
    for idx, column in enumerate(country_features_df.columns):
        if idx<9:
            # Create the plot
            ax = plt.subplot(len(country_features_df.columns)//3+1, 3, idx+1)
            
            # Get the x and y values if x is not infinite or nan
            
            x_all = country_features_df.loc[:,column].tolist()
            x = list()
            y = list()

            for i in range(len(x_all)):
                if (not np.isnan(x_all[i]) and not np.isinf(x_all[i])):
                    x.append(x_all[i])
                    y.append(y_all[i])            

            # Plot the original data
            ax.plot(x, y, 'o', label='original data')
            
            # Compute the linear regression 
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            
            # Plot the regression 
            ax.plot(np.array(x), intercept + slope*np.array(x), '--r', label='fitted line')
            ax.legend()
            ax.set_xlabel(column)
            ax.set_ylabel("Normalized number of tweets")
            plt.title("{} : r^2 {:.3f}".format(column, r_value**2) )
    fig.tight_layout()


def analyse_event(event_name, country_code, for_region = False):
    
    summary_tweets = load_tweets(event_name, country_code)
    
    # Compute the features for a specific event
    country_features_df = event_specific_features(country_code,summary_tweets)
    
    # Keep countries for which we have the real distance metric (i.e. for which we have all information)
    country_features_df = country_features_df.loc[~ country_features_df["real_distance"].isnull(),:]
    
    # If we only want to analyze for the corresponding region
    if for_region:
        # Extract the region
        region  = data.loc[country_code, "region"][0]
        country_features_df = country_features_df.loc[country_features_df["region"] == region,:]
    
    # Remove small countries from the model
    country_features_df = country_features_df.loc[country_features_df["POP"]>100000,:]
    country_features_df = country_features_df.loc[country_features_df["area"]>1000,:]
        
    # Compute and plot the correlations
    sns.heatmap(abs(country_features_df.corr("spearman")[["text", "Normalized Tweets"]]),annot=True)
    plot_correlations_with_regression(country_features_df)

Here we plot the analysis of the different events : 
- correlation between the raw number of tweets (called text) as well as the normalized number of tweets (by the estimated twitter baseline activity). 
- Linear regression for each of these metrics to see if they can be used to predict the output normalized number of tweets. 

One interesting observation to be made is that there are certain factors such as the GDP, population, number of internet users which exhibit relatively high correlations when observing the raw number of tweets. However this correlation disappears after normalization. 

This shows us that these parameters have a direct influence on the baseline twitter activity of each country. 


Remark : It is important to observe both the correlation factor and the regression!

In [ ]:
analyse_event("Bruxelles", "BE")

In [ ]:
analyse_event("Charlie-Hebdo", "FR")

In [ ]:
analyse_event("Istanbul", "TR")

In [ ]:
analyse_event("Lebanon", "LB")

In [ ]:
analyse_event("Nigeria_2015", "NG")

In [ ]:
analyse_event("Nigeria_2016", "NG")

In [ ]:
analyse_event("Orlando", "US")

In [ ]:
analyse_event("Pakistan", "PK")

Unfortunately, there are no factors which correlate highly and the R squared regression parameters is consistently low, regardless of the event. Often there are quite a large number of outlyers which are most likely due in part to the normalizing factor which is based on an estiamted baseline activity and may be quite far off from the truth. 


We attempted to see whether limiting our analysis to Europe for Charlie Hebdo and Brussels would improve the output but unfortunately that was not the case. 

In [ ]:
analyse_event("Charlie-Hebdo", "FR", for_region = True)

In [ ]:
analyse_event("Bruxelles", "BE",for_region = True)


The results vary little compared to what we had previously. This is not surprising as visually, even without knowledge of the interactions, simply observing what is going on around Europe we do not see the trends that we would expect in the number of tweets (refer to the interactive map on the tweet-awareness.eu website)

**TODO  : input a screenshot for charlie hebdo and brussels and explain what we would have expected to see**


## 4.2. Formatting and Standardizing the data

For the matrix factorization and the SVR it was first important to standardize the different features before combining them into one big feature vector. This was done for all the different metrics and the additional information for each of the countries. This was done at the start of the [SVR Model](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/SVR%20Model.ipynb) 
 notebook. The resulting dataframe is then exported under **all_features_pos.pickle** and **all_features_pos.pickle** (the latter to have only positive values for the matrix factorization)

## 4.3. Matrix Factorization Methods : LDA

As mentioned previously, we hoped that using topic detection methods would offer a good method to find optimal combinations of features which would link the countries together. Given a matrix of countries x topics we would have been able to compute the distance between the different vectors to construct our graph. Given the graph we could then use diffusion from a starting node and heat equations to see how the reaction would spread and then compare that to the outputs obtained for the events which were acquired. 


For details on how the matrix factorization was done, refer to the [Matrix Factorisation - LDA Model](https://github.com/LailaHms/ADA2017_Homeworks/blob/master/Project/AwarenessModel/Correlations%20and%20Linear%20Regression%20Model.ipynb) noteboook.

In [ ]:
W = pickle.load(open('AwarenessModel/LDA_weights.pickle','rb'))

distances = spatial.distance.squareform(spatial.distance.pdist(W)) # W
#plt.figure(figsize=[9,7])
#sns.heatmap(distances)

kernel_width = np.mean(distances)- np.std(distances)#distances.mean()

def kernel(x):
    return np.exp(- np.power(x/kernel_width,2))

# applying the kernel to the distances matrix :
weights = kernel(distances)
# setting the diagonal (self-loops) weights to 0 :
weights[range(len(weights)),range(len(weights))] = 0

# displaying a subset of the weights matrix :

plt.figure(figsize=[10,5])
sns.distplot(weights.flatten())
plt.figure(figsize=[9,7])
sns.heatmap(weights,cmap="Reds")


We can see that the matrix is relatively sparse with a limited number of countries with weights which are close to 1. 

In [ ]:
# Get the position of the nodes
c_data = pickle.load(open('DataEnriching/data.pickle','rb'))
c_data.drop("United States Minor Outlying Islands",inplace = True)

countries = c_data.index.tolist()

weights_df = pd.DataFrame(weights,columns=countries,index=countries)
weights_df.head()

Checking to see the closest countries to Germany to see if the output is coherent

In [ ]:
country = 'Germany'
weights_df[country].loc[weights_df[country] >= 0.9].sort_values(ascending = False)

####  Graph Construction and Graph Diffusion

Here we can already note quite a few incoherences as it is highly unlikely that an event in Germany would generate equal reactions in over 100 countries. 

Nonetheless we used heate diffusion on a graph to have a better visualization of the spread of reactions which we will then be able to see on the tweet-awareness.eu website

In [ ]:
# Get the position of the nodes
c_data = pickle.load(open('DataEnriching/data.pickle','rb'))
c_data.drop("United States Minor Outlying Islands",inplace = True)
countries = c_data.index.tolist()


weights_df = pd.DataFrame(weights,columns=countries,index=c_data.index.tolist())

country_pos = np.array(c_data[['latlng']].as_matrix())
country_pos = [ np.array(x[0]) for x in country_pos]
country_pos = np.vstack(country_pos)
country_pos = np.transpose(np.vstack((country_pos[:,1], country_pos[:,0])))

# Create the graph
G = graphs.Graph(weights_df.as_matrix())

G.set_coordinates(country_pos)
G.compute_fourier_basis()

# Create the heat diffusion filter
filt = filters.Heat(G, tau=100)


# Create the signal for a given country
for NODE, country in enumerate(weights_df.index):
    
    signal = np.zeros(G.N)
    signal[NODE] = 10

    # Apply the filter to the signal
    filtered_s = filt.filter(signal)
    
   
    if NODE > 0:
        filtered_signals = np.vstack((filtered_signals, filtered_s))
    else : 
        filtered_signals = filtered_s
    
    if country == "France":
        # Plot the signal on the graph
        G.plot_signal(filtered_s, vertex_size=50, highlight=NODE)

filtered_signals = np.transpose(filtered_signals)


Here we can see more clearly what is going on (with the dots placed at the center of each country). There does not seem to be any logic to the madness... 


In [ ]:
Image(filename=os.path.join(os.getcwd(), 'Visualization', "NMF_France.png"), width=1000)

In [ ]:
Image(filename=os.path.join(os.getcwd(), 'Visualization', "Charlie_hebdo.png"), width=1000)

The discrepancies speak for themselves. 

To visualize this for other countries please refer to the interactive plot on the data-story.

## 4.4. Regression Model To Predict Reaction Levels

We decided to attempt a regression model, knowing full well that we do not have enough datapoints to obtain pertinent results ("qui tente rien n'a rien"). In the present case we would like to have a model which would predict the reaction of all countries to an event in a specific country. The issue with this is that we need to have a multi-regressor which will compute a regression model for each of the countries. This means that we will have 247 regression models in total. To do so would have ideally required a minimum of 1000 events.

Unsurprisingly, the results were sub-par. 

## 4.5. Using Random Search To Optimize Mixing Weights

____
____
____

# 5. Critical Assessment and Conclusion

The goal of this project was to determine whether a set of simple metrics could be used to create a model which would adequately represent the links between countries and predict the reactions towards specific events on social media.


As we have shown the problem is far more complex than one would have hoped. Twitter is inherently biased because it is a social media plateform. This implies that people are not necessarily subscribed to the network, and if they are that they are active on network. In general social media is relatively new and tends to attract younger generations which are generally less aware about what is going on not only around the world but even in their immediate environment.


When people tweet they do not necessarily provide their location and this may also be cultural. Depending on the countries, people are more or less aware of the dangers of providing their locations on social media and therefore more or less inclined to provide personal information in general. This was partially corrected by using a normalizing factor with the tweets but is still an inherent bias in the original dataset.

The absence of reaction to an event on social media does not necessarily imply that people are not interested in what is going on. We assume that the proportion of people who react to an event is always the same with respect to the overall number of tweeters


Not having standardized locations required creating a mapping which in itself is imperfect and implies loss of a given amount of data. 


Our metrics are also imperfect and can difficultly model on their own a complex system which is highly linked to social and behavioural aspects. 


Furthermore, creating a model requires much more data than what we had at hand, both in terms of number of tweets and in terms of events analyzed. 


For all of these reasons it was evident that creating a metric would be challenging. In general it would seem that we need much more complex pararmeters to describe our system than the ones we have currently in order to correclty predict what is going on especially since awareness does not necessarily imply reaction.


**TODO / NOTE TO BE CONTEMPLATED FOR CONCLUSION TO THE WORST: to see if the fact that the country is very large with a large population that can diminish the interest compared to what is happening elsewhere in the world**

# 6. Data Story



For the Data story we have decided to use the possibilities of Github Pages and Jekyll in order to create a beautiful Data Story. We used for that a template exploiting Bootstrap : [GrayScale](https://github.com/jeromelachaud/grayscale-theme). 

For the Interactive Maps visualization, we used several tools written in Javascript, including  [GrayScale](https://github.com/jeromelachaud/grayscale-theme). 

For the Interactive Maps visualization, we used several tools written in Javascript, including D3.js , Topojson.js  and adional D3 extensions. 

Enjoy 🙂 

[Tweet Awareness](http://tweet-awareness.eu).


# 7. Contributions

We summarized the contributions of each of the members in the following table where green refers to major contributions and yellow to minor contributions. 

In [ ]:
Image(filename=os.path.join(os.getcwd(), 'Visualization', "member_contributions.png"), width=700)